In [1]:
import pubstats.__init__ as ps
import requests
import pandas as pd
import numpy as np
import json
import difflib
import time
import sys
import os
from bs4 import BeautifulSoup
# from py2cytoscape import cyrest
import subprocess as sb
from subprocess import Popen, PIPE, STDOUT
import matplotlib
import matplotlib.pyplot as plt
import paramiko
import html

In [4]:
pmc="/nexus/posix0/MAGE-flaski/service/projects/data/Bioinformatics/bit_gpt_age/pmc"
pbmc_folders=os.listdir(pmc)
pbmc_folders=[ s for s in pbmc_folders if os.path.isdir(f"{pmc}/{s}") ] 
pmc_files=pd.DataFrame()
for f in  pbmc_folders:
    print(f)
    sys.stdout.flush()
    files=os.listdir(f"{pmc}/{f}")
    pmc_files_=pd.DataFrame()
    for f_ in files:
        content=[]
        with open(f"{pmc}/{f}/{f_}", "r" ) as txt :
            for l in txt:
                l=l.split("\t")[0]
                content.append(l)
        target=f_.replace("filelist.txt","tar.gz") 
        if f == "manuscript":
            target=f"https://ftp.ncbi.nlm.nih.gov/pub/pmc/manuscript/txt/{target}"
        else: 
            target=f"https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/{f}/txt/{target}"
        tmp=pd.DataFrame({"folder":f,"file":f_, "files":content, "tar.gz": target  })
        pmc_files_=pd.concat([pmc_files_,tmp])
    pmc_files_.to_csv(f"{pmc}/{f}.tsv", sep="\t",index=None)
    pmc_files=pd.concat([pmc_files,pmc_files_])
    
pmc_files=pmc_files[pmc_files["files"]!="Article File"]
pmc_files["subfolder"]=pmc_files["files"].apply(lambda x: x.split("/")[0])
def get_ftp_pmc(x):
    try:
        r=x.split("/")[1].split(".txt")[0] 
    except:
        print(x)
        r=x.split("/")[1].split(".txt")[0] 
    return r

pmc_files["pmc"]=pmc_files["files"].apply(lambda x: get_ftp_pmc(x) )
pmc_files.head()

oa_noncomm
oa_comm
manuscript
oa_other


,folder,file,files,tar.gz,subfolder,pmc
1,oa_noncomm,oa_noncomm_txt.incr.2023-06-21.filelist.txt,PMC009xxxxxx/PMC9770323.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9770323
2,oa_noncomm,oa_noncomm_txt.incr.2023-06-21.filelist.txt,PMC009xxxxxx/PMC9882630.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9882630
3,oa_noncomm,oa_noncomm_txt.incr.2023-06-21.filelist.txt,PMC009xxxxxx/PMC9907098.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907098
4,oa_noncomm,oa_noncomm_txt.incr.2023-06-21.filelist.txt,PMC009xxxxxx/PMC9907117.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907117
5,oa_noncomm,oa_noncomm_txt.incr.2023-06-21.filelist.txt,PMC009xxxxxx/PMC9907125.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907125


In [5]:
tmp=pmc_files[pmc_files["pmc"]=="PMC6225988"]
print(tmp["tar.gz"].tolist())
tmp

['https://ftp.ncbi.nlm.nih.gov/pub/pmc/manuscript/txt/author_manuscript_txt.PMC006xxxxxx.baseline.2023-06-16.tar.gz', 'https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_other/txt/oa_other_txt.PMC006xxxxxx.baseline.2023-06-17.tar.gz']


,folder,file,files,tar.gz,subfolder,pmc
21597,manuscript,author_manuscript_txt.PMC006xxxxxx.baseline.20...,PMC006xxxxxx/PMC6225988.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/manuscrip...,PMC006xxxxxx,PMC6225988
4436,oa_other,oa_other_txt.PMC006xxxxxx.baseline.2023-06-17....,PMC006xxxxxx/PMC6225988.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC006xxxxxx,PMC6225988


In [3]:
BASEURL="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
ESEARCH=BASEURL+"esearch.fcgi?db=pubmed&rettype=json&retmode=json&term="
ESUMMARY=BASEURL+"esummary.fcgi?db=pubmed&rettype=json&retmode=json&id="

title="Sequencing the Immunoglobulin Heavy-Chain Locus (IgH) in Turquoise Killifish ( Nothobranchius furzeri)"

def get_pubmedid(title,author,doi,ESEARCH=ESEARCH ):
    
    if doi:
        url=ESEARCH+doi#+"[Title]"
        r = requests.post(url = url, verify=False)
        r = json.loads(r.content)
        data=r['esearchresult']
        idlist=data['idlist']
        if len(idlist) == 1:
            return idlist[0]
        
    url=ESEARCH+'"'+title+'"'#+"[Title]"
    r = requests.post(url = url, verify=False)
    r = json.loads(r.content)
    data=r['esearchresult']
    idlist=data['idlist']
    if len(idlist) != 1:
        url=url+" "+author
        r = requests.post(url = url, verify=False)
        r = json.loads(r.content)
        data=r['esearchresult']
        idlist=data['idlist']
        if len(idlist) != 1:
            print(len(idlist), doi, title)
            return None
        else:
            return idlist[0]

    else:
        return idlist[0]
        
    
# print(idlist)
# for article_id in idlist:
#     url = esummary+article_id
#     r = requests.post(url = url, verify=False)
#     r = json.loads(r.content)

In [4]:
get_pubmedid("ncreasing number of long-lived ancestors marks a decade of healthspan extension and healthier metabolomics profiles", "van den Berg, Niels", "10.1038/s41467-023-40245-6")

'37500622'

In [5]:
[ "Deletion of endogenous Tau proteins is not detrimental in Drosophila", "26976084", "10.1038/srep23102"],\
[ "Dietary Restriction: Theory Fails to Satiate - Response",

SyntaxError: incomplete input (833280383.py, line 2)

In [6]:
# pubmedids=[]
# pubmedid=None
# title=None
# author=None
# titleid=None
# doi=None
# with open("export_age_all_publ_20230817.txt", "r") as f:
#     for l in f.readlines():
#         if ( l[:2] == "%A" ) and ( not author ):
#             author=l[3:].split("\n")[0] #-4]
#         if l[:9] == '%F OTHER:':
#             pubmedid=l.split("OTHER: ")[1].split("\n")[0]
#         if l[:2] == "%T" :
#             title=l[3:].split("\n")[0]   #-4]
#         if l[:2] == "%R" :
#             doi=l[3:].split("\n")[0]   
#         if ( l == "\n" ) and title:
#             # print(pubmedid,titleid,doi, title )
#             if not pubmedid:
#                 pubmedid=get_pubmedid(title, author,doi)
#             pubmedids.append([pubmedid,doi, title ])
#             pubmedid=None
#             title=None
#             titleid=None
#             author=None    
#             doi=None

In [7]:
# pubmedids=pd.DataFrame(pubmedids,columns=["PMID","DOI","Title"])
# pubmedids.columns=["PMID","DOI","Title"]
# pubmedids.head()
# pubmedids.to_csv("pubmedids.tsv", sep="\t",index=None)

In [9]:
pubmedids=pd.read_csv("pubmedids.tsv",sep="\t")
pubmedids.head()

,PMID,DOI,Title
0,NaN,NaN,"lllumination Power, Stability, and Linearity M..."
1,31636117,10.1083/jcb.201902069,Soluble tubulin is significantly enriched at m...
2,34302049,NaN,Monitoring the point spread function for quali...
3,37563253,10.1038/s41556-023-01198-6,Malonyl-CoA is a conserved endogenous ATP-comp...
4,37384530,10.1016/j.celrep.2023.112722,Dietary restriction mitigates the age-associat...


In [10]:
ids=pubmedids.dropna(subset=["PMID"])
noids=pubmedids[ ~pubmedids.index.isin(ids.index.tolist() ) ]
print(len(noids))
ids_values=ids["PMID"].tolist()
noids

34


,PMID,DOI,Title
0,NaN,NaN,"lllumination Power, Stability, and Linearity M..."
12,NaN,10.1103/PhysRevX.5.041014,Strongly Deterministic Population Dynamics in ...
20,NaN,10.18632/aging.203859,Meeting Report: Aging Research and Drug Discov...
57,NaN,NaN,Hypoxia promotes osteogenesis via regulation o...
74,NaN,10.1101/2022.12.10.519872,Comprehensive Evaluation of Rapamycin’s Specif...
75,NaN,10.1101/2022.11.06.515351,Malonyl-CoA is an ancient physiological ATP-co...
90,NaN,NaN,Substrate recognition by the AAA plus chaperon...
275,NaN,NaN,Announcements from your Editorial Team :
317,NaN,10.1038/srep23102http://www.nature.com/article...,Deletion of endogenous Tau proteins is not det...
354,NaN,DOI: 10.1126/science.329.5995.1015,Dietary Restriction: Theory Fails to Satiate -...


In [11]:
pmc_files.head()

,folder,files,tar.gz,subfolder,pmc
1,oa_noncomm,PMC009xxxxxx/PMC9770323.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9770323
2,oa_noncomm,PMC009xxxxxx/PMC9882630.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9882630
3,oa_noncomm,PMC009xxxxxx/PMC9907098.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907098
4,oa_noncomm,PMC009xxxxxx/PMC9907117.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907117
5,oa_noncomm,PMC009xxxxxx/PMC9907125.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907125


In [15]:
# article_id=ids_values[0]
# print(article_id)
article_ids=[]
for article_id in ids_values:
    # works=False
    # while not works:
    try:
        url=ESUMMARY+article_id
        r = requests.post(url = url, verify=False)
        r = json.loads(r.content)
        data=r['result'][article_id]
        values=data["articleids"]
        pmc=[ s["value"] for s in values if s["idtype"]=="pmc" ]
        pmcid=[ s["value"] for s in values if s["idtype"]=="pmcid" ]
        if len(pmc) == 1 :
            r=pmc[0]
        elif len(pmcid) == 1:
            r=pmc[0]
        else:
            r=None
        article_ids.append([ article_id, r] )
    except:
        print("failed for ", url)
        # time.sleep(2)
        sys.stdout.flush()
        
article_ids=pd.DataFrame(article_ids, columns=["PMID","PMCID"])

failed for  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&rettype=json&retmode=json&id=WOS:000308098200011
failed for  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&rettype=json&retmode=json&id=632395508
failed for  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&rettype=json&retmode=json&id=WOS:000268375500023
failed for  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&rettype=json&retmode=json&id=WOS:000466360300001
failed for  https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&rettype=json&retmode=json&id=WOS:000472611700005


In [17]:
pubmedids=pd.merge(pubmedids,article_ids,on=["PMID"],how="left")
pubmedids.to_csv("pubmedids.tsv", sep="\t",index=None)

In [6]:
pubmedids=pd.read_csv("pubmedids.tsv",sep="\t")
pubmedids.head()

,PMID,DOI,Title,PMCID
0,NaN,NaN,"lllumination Power, Stability, and Linearity M...",NaN
1,31636117,10.1083/jcb.201902069,Soluble tubulin is significantly enriched at m...,PMC6891098
2,34302049,NaN,Monitoring the point spread function for quali...,PMC8302645
3,37563253,10.1038/s41556-023-01198-6,Malonyl-CoA is a conserved endogenous ATP-comp...,NaN
4,37384530,10.1016/j.celrep.2023.112722,Dietary restriction mitigates the age-associat...,PMC10391628


In [7]:
bash='''
#!/bin/bash

cd /nexus/posix0/MAGE-flaski/service/projects/data/Bioinformatics/bit_gpt_age/
mkdir -p pmc_papers
cd pmc_papers

#################


'''

pubmedids_valid=pubmedids[["PMCID"]].dropna()["PMCID"].tolist()

papers=pmc_files.dropna(subset=["tar.gz"])
papers=papers[papers["pmc"].isin(pubmedids_valid)]
print(len(papers))

for target in list( set(papers["tar.gz"].tolist() )) :
    if target:
        wget=f"wget {target}\n"

        bash=bash+wget

        files=papers[papers["tar.gz"]==target]["files"].tolist()
        files=list(set(files))
        files=" ".join(files)
        targz=target.split("/")[-1]
        untar=f"tar -zxvf {targz} {files}\n"

        bash=bash+untar

        rm=f"rm -rf {targz}\n"

        ash="\n#################\n\n"

        bash=bash+rm+ash

print(bash)

with open("download.extract.pmc.sh", "w" ) as sh:
    sh.write(bash)
        

634

#!/bin/bash

cd /nexus/posix0/MAGE-flaski/service/projects/data/Bioinformatics/bit_gpt_age/
mkdir -p pmc_papers
cd pmc_papers

#################


wget https://ftp.ncbi.nlm.nih.gov/pub/pmc/manuscript/txt/author_manuscript_txt.incr.2023-08-02.tar.gz
tar -zxvf author_manuscript_txt.incr.2023-08-02.tar.gz PMC010xxxxxx/PMC10388377.txt
rm -rf author_manuscript_txt.incr.2023-08-02.tar.gz

#################

wget https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_other/txt/oa_other_txt.PMC004xxxxxx.baseline.2023-06-17.tar.gz
tar -zxvf oa_other_txt.PMC004xxxxxx.baseline.2023-06-17.tar.gz PMC004xxxxxx/PMC4858780.txt PMC004xxxxxx/PMC4250049.txt PMC004xxxxxx/PMC4338562.txt PMC004xxxxxx/PMC4161471.txt
rm -rf oa_other_txt.PMC004xxxxxx.baseline.2023-06-17.tar.gz

#################

wget https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/txt/oa_comm_txt.incr.2023-07-12.tar.gz
tar -zxvf oa_comm_txt.incr.2023-07-12.tar.gz PMC010xxxxxx/PMC10328083.txt
rm -rf oa_comm_txt.incr.2023-07-12.tar.gz

#####

In [19]:
pmc_files.head()

,folder,files,tar.gz,subfolder,pmc
1,oa_noncomm,PMC009xxxxxx/PMC9770323.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9770323
2,oa_noncomm,PMC009xxxxxx/PMC9882630.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9882630
3,oa_noncomm,PMC009xxxxxx/PMC9907098.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907098
4,oa_noncomm,PMC009xxxxxx/PMC9907117.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907117
5,oa_noncomm,PMC009xxxxxx/PMC9907125.txt,https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/o...,PMC009xxxxxx,PMC9907125
